In [186]:
import json
import numpy as np

__Open JSON__

In [187]:
json_file = open("example-2layer.json", "r") # open json file in reading mode
data = json.load(json_file) # load json file in datastructure with json

__Save weights to matrices__

In [188]:
matrices = [] # empty list for all matrices
for layer in data: # iterate over layers in json file
    
    matrix = np.zeros([int(data[layer]["size_out"]), int(data[layer]["size_in"])], dtype = float) # create empty matrix

    for index, node in enumerate(data[layer]["weights"]): # iterate over nodes in layer

        for weight in data[layer]["weights"][node]: # iterate over weights in node
            
            matrix[int(weight) - 1][int(node) - 1] = float(data[layer]["weights"][node][weight])
            # add weight to correct place in matrix
            
    matrices.append(matrix) # add matrix to all matrices

In [189]:
matrixinput = np.matrix([[1]] * int(data["layer1"]["size_in"])) # input matrix for the neural network
matrices.reverse() # reverse matrices to do L0 * L1 * L2... instead of L2 * L1 * L0
new_matrix = matrices[0] # set first starting matrix to be multiplied with

__Matrix math__

In [190]:
for layer, matrix in enumerate(matrices): # iterate over matrices
    print("Matrix "+str(layer + 1)+":\n"+str(matrix)+'\n') # print matrix
     
    if len(matrices) == 1: # if only 1 matrix available, multiply this matrix with input vector
        matrix2 = matrixinput
    elif layer + 1 == len(matrices): # if all matrices are multiplied, multiply last product with input vector
        matrix = new_matrix
        matrix2 = matrixinput
    else: # else multiply all matrices with each other starting with new_matrix as matrices[0]
        matrix = new_matrix
        matrix2 = matrices[layer + 1]

    new_matrix = np.zeros([len(matrix), len(matrix2[0])], dtype = float) # create new empty matrix
    
    for irow, row in enumerate(matrix): # iterate over rows in matrix
        
        for i in range(0, len(matrix2[0])): # iterate over number of columns in matrixvar
            som = 0
        
            for iweight, weight in enumerate(row): # iterate over weights in row
                som += matrix[irow][iweight] * matrix2[iweight][i] # add sum of muliplied weights to variable sum 
            
            new_matrix[irow][i] = som # add sum to correct place in newmatrix
    
print("Output values:\n", new_matrix) # print output

Matrix 1:
[[ 0.5  0.2 -0.1  0.9]
 [ 0.2 -0.5  0.3  0.1]]

Matrix 2:
[[ 0.5  0.2  0.   0.  -0.2]
 [ 0.2 -0.5 -0.1  0.9 -0.8]
 [ 0.   0.2  0.   0.1 -0.1]
 [ 0.1  0.8  0.3  0.  -0.7]]

Output values:
 [[0.62]
 [0.36]]
